<a href="https://colab.research.google.com/github/Brandt-DSTI/Computer_Vision_ISIC_2024/blob/main/Copy_of_SN_BES_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Chunk 0: Mount Google Drive and Copy Files
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
drive_hdf5_path = '/content/drive/MyDrive/isic-2024-challenge/train-image.hdf5'
drive_metadata_path = '/content/drive/MyDrive/isic-2024-challenge/train-metadata.csv'
local_hdf5_path = '/content/train-image.hdf5'
local_metadata_path = '/content/train-metadata.csv'

# Copy files from Drive to local Colab session
shutil.copy(drive_hdf5_path, local_hdf5_path)
shutil.copy(drive_metadata_path, local_metadata_path)

print("Files copied to local Colab session.")
print(f"Train HDF5 file exists: {os.path.exists(local_hdf5_path)}")
print(f"Train metadata file exists: {os.path.exists(local_metadata_path)}")

Mounted at /content/drive
Files copied to local Colab session.
Train HDF5 file exists: True
Train metadata file exists: True


In [ ]:
# Chunk 1: Setup and Imports

# Install required packages
!pip install albumentations timm tqdm h5py opencv-python-headless imbalanced-learn mealpy

# Imports
import os
import random
import time
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import logging

from PIL import Image
import io
import h5py
import cv2

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau
from torch.amp import autocast, GradScaler

from mealpy import FloatVar, BES
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, f1_score, roc_curve, auc

import timm
from imblearn.over_sampling import RandomOverSampler

import albumentations as A
from albumentations.pytorch import ToTensorV2

import json
from datetime import datetime

import torchvision.models as models

# Setup logging
logging.basicConfig(filename='optimization.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Random seed setup for reproducibility
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

# Load metadata
df_train = pd.read_csv(local_metadata_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 74.3 MB/s eta 0:00:00
Using device: cuda
GPU: NVIDIA L4
Number of GPUs: 1


<ipython-input-2-e64251c63e1e>:67: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(local_metadata_path)


In [ ]:
# Chunk 2: Define Dataset and Augmentation

class ISICDataset(Dataset):
    def __init__(self, hdf5_file, isic_ids, targets=None, transform=None, vsurf_features=None):
        self.hdf5_file = h5py.File(hdf5_file, 'r')
        self.isic_ids = isic_ids
        self.targets = targets
        self.transform = transform
        self.vsurf_features = vsurf_features
        self.valid_indices = self._get_valid_indices()

    def _get_valid_indices(self):
        valid_indices = []
        for idx in range(len(self.isic_ids)):
            try:
                img_bytes = self.hdf5_file[self.isic_ids[idx]][()]
                Image.open(io.BytesIO(img_bytes))
                valid_indices.append(idx)
            except Exception as e:
                print(f"Error processing image at index {idx}: {e}")
        return valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_bytes = self.hdf5_file[self.isic_ids[real_idx]][()]
        img = Image.open(io.BytesIO(img_bytes))
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        vsurf_feat = self.vsurf_features[real_idx] if self.vsurf_features is not None else np.zeros(5)
        target = self.targets[real_idx] if self.targets is not None else None
        return img, vsurf_feat, target

    def __del__(self):
        self.hdf5_file.close()

def get_augmentation(is_training=True):
    if is_training:
        return A.Compose([
            A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0)),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.CoarseDropout(max_holes=8, max_height=8, max_width=8, min_holes=5, min_height=8, min_width=8, fill_value=0, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(224, 224),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

In [ ]:
# Chunk 3: Train-Validation-Test Split, Random Oversampling, and DataLoader

def perform_oversampling(df_train, vsurf_features=None):
    X = df_train['isic_id'].values.reshape(-1, 1)
    y = df_train['target'].values
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X, y)

    resampled_vsurf_features = vsurf_features[ros.sample_indices_] if vsurf_features is not None else None

    resampled_df = pd.DataFrame({
        'isic_id': X_resampled.flatten(),
        'target': y_resampled
    })

    return resampled_df, resampled_vsurf_features

# Split data into train (70%), validation (15%), and test (15%)
train_df, temp_df = train_test_split(df_train, test_size=0.3, stratify=df_train['target'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['target'], random_state=42)

print(f"Train set: {len(train_df)} samples, Validation set: {len(val_df)} samples, Test set: {len(test_df)} samples")

def get_dataloaders(df, hdf5_file_path, batch_size, vsurf_features=None, is_training=True):
    dataset = ISICDataset(hdf5_file_path,
                          df['isic_id'].values,
                          df['target'].values,
                          transform=get_augmentation(is_training),
                          vsurf_features=vsurf_features)
    return DataLoader(dataset, batch_size=batch_size, shuffle=is_training, num_workers=4, pin_memory=True)

Train set: 280741 samples, Validation set: 60159 samples, Test set: 60159 samples


In [ ]:
# Chunk 4: Model Setup, Checkpointing Functions, and Objective Function with VSURF features
# JSON logging of hyperparameters for easier sharing of outputs

class SqueezeNetWithVSURF(nn.Module):
    def __init__(self, num_classes=1, vsurf_size=5, dropout_rate=0.5):
        super(SqueezeNetWithVSURF, self).__init__()

        # Load pre-trained SqueezeNet
        self.squeezenet = models.squeezenet1_1(pretrained=True)

        # Replace the last convolutional layer
        self.squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)

        # Additional layers for VSURF features
        self.vsurf_fc = nn.Sequential(
            nn.Linear(vsurf_size, 32),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        # Combine features
        self.final_fc = nn.Linear(num_classes + 32, num_classes)

    def forward(self, x, vsurf):
        # SqueezeNet forward pass
        x = self.squeezenet(x)
        x = x.view(x.size(0), -1)

        # VSURF features forward pass
        vsurf = self.vsurf_fc(vsurf)

        # Combine features
        combined = torch.cat((x, vsurf), dim=1)

        # Final classification
        out = self.final_fc(combined)

        return out

def setup_model(num_classes=1, vsurf_size=5, dropout_rate=0.5):
    model = SqueezeNetWithVSURF(num_classes=num_classes, vsurf_size=vsurf_size, dropout_rate=dropout_rate)
    return model.to(device)

def save_checkpoint(epoch, model, optimizer, learning_rate, batch_size, dropout_rate):
    os.makedirs('bes_checkpoints', exist_ok=True)

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'learning_rate': learning_rate,
        'batch_size': batch_size,
        'dropout_rate': dropout_rate
    }
    checkpoint_path = f"bes_checkpoints/bes_checkpoint_epoch_{epoch}.pth"
    torch.save(checkpoint, checkpoint_path)
    logging.info(f"Checkpoint saved: {checkpoint_path}")

def load_latest_checkpoint():
    if not os.path.exists("bes_checkpoints"):
        logging.info("Checkpoint directory does not exist, starting from scratch.")
        return None

    checkpoints = [f for f in os.listdir("bes_checkpoints") if f.startswith("bes_checkpoint_epoch_")]
    if not checkpoints:
        logging.info("No checkpoints found, starting from scratch.")
        return None

    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    checkpoint_path = os.path.join("bes_checkpoints", latest_checkpoint)
    checkpoint = torch.load(checkpoint_path)

    model = setup_model(num_classes=1, dropout_rate=checkpoint['dropout_rate'])
    model.load_state_dict(checkpoint['model_state_dict'])

    optimizer = Adam(model.parameters(), lr=checkpoint['learning_rate'], weight_decay=1e-5)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    logging.info(f"Loaded checkpoint: {checkpoint_path}")
    return checkpoint, model, optimizer

def train_and_evaluate_model(model, optimizer, scheduler, train_loader, val_loader, num_epochs=10, patience=5):
    criterion = nn.BCEWithLogitsLoss()
    scaler = GradScaler()
    best_val_f1 = 0
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, vsurf_features, targets in train_loader:
            images = images.to(device)
            vsurf_features = vsurf_features.to(device).float()  # Ensure float type
            targets = targets.to(device).float().view(-1, 1)

            optimizer.zero_grad()
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(images, vsurf_features)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        val_f1 = evaluate_model(model, val_loader)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val F1: {val_f1:.4f}")

        scheduler.step(val_f1)

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            epochs_without_improvement = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

    return best_val_f1

@torch.no_grad()
def evaluate_model(model, val_loader):
    model.eval()
    all_targets = []
    all_outputs = []

    for images, vsurf_features, targets in val_loader:
        images = images.to(device)
        vsurf_features = vsurf_features.to(device).float()  # Ensure float type
        outputs = model(images, vsurf_features)

        all_outputs.extend(torch.sigmoid(outputs).cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

    binary_outputs = (np.array(all_outputs) > 0.5).astype(int)
    val_f1 = f1_score(all_targets, binary_outputs)

    return val_f1

def log_hyperparameters(hyperparameters, metrics, model_name, dataset_info):
    log = {
        "timestamp": datetime.now().isoformat(),
        "model": model_name,
        "dataset": dataset_info,
        "hyperparameters": hyperparameters,
        "performance_metrics": metrics
    }

    # Save as JSON for easy reading and sharing
    with open('hyperparameters_log.json', 'w') as f:
        json.dump(log, f, indent=4)

    # Also save as .pth for PyTorch compatibility
    torch.save(log, 'hyperparameters_log.pth')

def objective_function(solution):
    try:
        learning_rate, dropout_rate = solution
        batch_size = 64  # Fixed batch size, you can make this a parameter if needed

        logging.info(f"Trying solution: LR={learning_rate:.6f}, DR={dropout_rate:.2f}")

        train_df, val_df = train_test_split(df_train, test_size=0.2, stratify=df_train['target'], random_state=42)

        vsurf_columns = ['clin_size_long_diam_mm', 'tbp_lv_H', 'tbp_lv_deltaLBnorm', 'tbp_lv_perimeterMM', 'tbp_lv_Hext']
        vsurf_features = df_train[vsurf_columns].values

        df_train_resampled, resampled_vsurf_features = perform_oversampling(train_df, vsurf_features=vsurf_features[train_df.index])

        train_loader = get_dataloaders(df_train_resampled, local_hdf5_path, batch_size, vsurf_features=resampled_vsurf_features)
        val_loader = get_dataloaders(val_df, local_hdf5_path, batch_size, vsurf_features=vsurf_features[val_df.index], is_training=False)

        model = setup_model(num_classes=1, dropout_rate=dropout_rate, vsurf_size=5)
        optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

        best_val_f1 = train_and_evaluate_model(model, optimizer, scheduler, train_loader, val_loader, num_epochs=3)

        # Prepare data for logging
        best_hyperparameters = {
            'learning_rate': learning_rate,
            'dropout_rate': dropout_rate
        }

        metrics = {
            'val_f1': best_val_f1,
        }

        dataset_info = {
            "name": "ISIC 2024 Challenge",
            "total_samples": len(df_train),
            "benign_samples": len(df_train[df_train['target'] == 0]),
            "malignant_samples": len(df_train[df_train['target'] == 1])
        }

        # Log hyperparameters
        log_hyperparameters(best_hyperparameters, metrics, "SqueezeNet", dataset_info)

        # Save the current best solution
        current_best = {
            'learning_rate': learning_rate,
            'dropout_rate': dropout_rate,
            'val_f1': best_val_f1
        }
        torch.save(current_best, 'current_best_hyperparameters.pth')

        logging.info(f"Solution performance: F1={best_val_f1:.4f}")

        return -best_val_f1  # We're minimizing, so return negative F1 score

    except Exception as e:
        logging.error(f"Error in objective function: {str(e)}")
        return float('inf')  # Return large value to indicate failure


In [ ]:
# Chunk 5: BES Optimization Setup

# Define the problem dictionary
problem_dict = {
    "obj_func": objective_function,
    "bounds": FloatVar(
        lb=[1e-5, 0.1],  # Lower bounds: learning_rate, dropout_rate
        ub=[1e-3, 0.5],  # Upper bounds: learning_rate, dropout_rate
        name=["learning_rate", "dropout_rate"]
    ),
    "minmax": "min",  # We want to minimize the objective function (negative F1 score)
}

# BES Optimizer Setup
optimizer = BES.OriginalBES(
    epoch=50,
    pop_size=10,
    a_factor=10,
    R_factor=1.5,
    alpha=2.0,
    c1=2.0,
    c2=2.0
)

# Set a callback function to save checkpoints after each epoch
def checkpoint_callback(epoch, population):
    os.makedirs('bes_checkpoints', exist_ok=True)
    checkpoint = {
        'epoch': epoch,
        'population': population
    }
    checkpoint_path = f"bes_checkpoints/bes_checkpoint_epoch_{epoch}.pth"
    torch.save(checkpoint, checkpoint_path)
    logging.info(f"Checkpoint saved: {checkpoint_path}")

optimizer.callback = checkpoint_callback

# Define termination conditions
term_dict = {
    "max_epoch": 50,
    "max_fe": 1000,
    "max_time": 14400,  # 4 hours in seconds
    "max_early_stop": 30
}

# Check for an existing checkpoint to resume optimization
latest_checkpoint = load_latest_checkpoint()
if latest_checkpoint:
    start_epoch = latest_checkpoint['epoch']
    optimizer.g_best = latest_checkpoint['g_best']
    optimizer.pop = latest_checkpoint['pop']
    optimizer.epoch = start_epoch
    logging.info(f"Resuming optimization from epoch {start_epoch}")
else:
    start_epoch = 0
    logging.info("Starting new optimization process")

# Solve the optimization problem using BES
try:
    best_solution, best_fitness = optimizer.solve(problem_dict, termination=term_dict)

    logging.info("Optimization completed successfully.")
    logging.info(f"Best Hyperparameters: Learning Rate = {best_solution[0]:.6f}, "
                 f"Dropout Rate = {best_solution[1]:.2f}")
    logging.info(f"Best Validation F1 Score: {-best_fitness:.4f}")

    # Output the results
    print("Optimization completed.")
    print(f"Best Hyperparameters: Learning Rate = {best_solution[0]:.6f}, "
          f"Dropout Rate = {best_solution[1]:.2f}")
    print(f"Best Validation F1 Score: {-best_fitness:.4f}")

except Exception as e:
    logging.error(f"Error during optimization process: {str(e)}")
    print(f"An error occurred during optimization. Check the log file for details.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 103MB/s]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning ra

Epoch 1/3, Train Loss: 0.5397, Val F1: 0.0106
Epoch 2/3, Train Loss: 0.4367, Val F1: 0.0117
Epoch 3/3, Train Loss: 0.4160, Val F1: 0.0124


INFO:mealpy.swarm_based.BES.OriginalBES:Solving single objective optimization problem.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/3, Train Loss: 0.8841, Val F1: 0.0088
Epoch 2/3, Train Loss: 0.4316, Val F1: 0.0150
Epoch 3/3, Train Loss: 0.3563, Val F1: 0.0144


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Epoch 1/3, Train Loss: nan, Val F1: 0.0000
Epoch 2/3, Train Loss: nan, Val F1: 0.0000
Epoch 3/3, Train Loss: nan, Val F1: 0.0000


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/3, Train Loss: 0.3944, Val F1: 0.0267
Epoch 2/3, Train Loss: 0.0807, Val F1: 0.0822
Epoch 3/3, Train Loss: 0.0442, Val F1: 0.0657


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Chunk 6: Model evaluation with Kaggle pAUC scoring
import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, classification_report

def competition_score(y_true, y_pred, min_tpr=0.80):
    # Rescale the target. Set 0s to 1s and 1s to 0s
    v_gt = abs(np.asarray(y_true) - 1)
    # Flip the predictions to their complements
    v_pred = -1.0 * np.asarray(y_pred)
    max_fpr = abs(1 - min_tpr)
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError(f"Expected min_tpr in range [0, 1), got: {min_tpr}")
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return partial_auc, fpr, tpr

@torch.no_grad()
def evaluate_model(model, test_loader, device):
    model.eval()
    all_predictions = []
    all_targets = []
    for inputs, vsurf_features, targets in tqdm(test_loader, desc="Evaluating"):
        inputs = inputs.to(device)
        vsurf_features = vsurf_features.to(device).float()
        outputs = model(inputs, vsurf_features)
        predictions = torch.sigmoid(outputs).cpu().numpy()
        all_predictions.append(predictions)
        all_targets.append(targets.numpy())

    all_predictions = np.concatenate(all_predictions).flatten()
    all_targets = np.concatenate(all_targets)

    # Print diagnostic information
    print(f"Predictions - Min: {all_predictions.min():.4f}, Max: {all_predictions.max():.4f}, Mean: {all_predictions.mean():.4f}")
    print(f"Unique prediction values: {len(np.unique(all_predictions))}")
    print(f"Prediction distribution:\n{np.histogram(all_predictions, bins=10)}")
    print(f"Target distribution: {np.bincount(all_targets)}")

    # Calculate metrics
    try:
        auc_score = roc_auc_score(all_targets, all_predictions)
        pauc_score, fpr, tpr = competition_score(all_targets, all_predictions, min_tpr=0.80)
        print(f"ROC curve points: {len(fpr)}")
        print(f"TPR range: {tpr.min():.4f} to {tpr.max():.4f}")
    except Exception as e:
        print(f"Warning: AUC calculation failed. Error: {str(e)}")
        print("Setting AUC and pAUC to minimum values.")
        auc_score = 0.5
        pauc_score = 0.0
        fpr, tpr = None, None

    binary_predictions = (all_predictions > 0.5).astype(int)
    f1 = f1_score(all_targets, binary_predictions)

    print(f"AUC: {auc_score:.4f}")
    print(f"pAUC (competition metric): {pauc_score:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_targets, binary_predictions))

    return auc_score, pauc_score, f1

# Test the evaluate_model function
print("Testing evaluate_model function:")
# Load the best model (assuming it's already trained and saved)
best_model = setup_model(num_classes=1, vsurf_size=5, dropout_rate=best_hyperparameters['dropout_rate'])
best_model.load_state_dict(torch.load('best_model.pth'))
best_model = best_model.to(device)

# Prepare the test dataset
test_vsurf_features = test_df[vsurf_columns].values

print("Test VSURF features shape:", test_vsurf_features.shape)
print("Test VSURF features type:", type(test_vsurf_features))
print("Test VSURF features dtype:", test_vsurf_features.dtype)

# Create a test loader for the test set
test_loader = get_dataloaders(test_df, local_hdf5_path, batch_size=64, vsurf_features=test_vsurf_features, is_training=False)

# Run the evaluation
test_auc_score, test_pauc_score, test_f1_score = evaluate_model(best_model, test_loader, device)
print(f"Test AUC: {test_auc_score:.4f}")
print(f"Test pAUC (competition metric): {test_pauc_score:.4f}")
print(f"Test F1 Score: {test_f1_score:.4f}")

Testing evaluate_model function:
Test VSURF features shape: (60159, 5)
Test VSURF features type: <class 'numpy.ndarray'>
Test VSURF features dtype: float64


<ipython-input-18-2681bc53a174>:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_model.pth'))


Evaluating:   0%|          | 0/940 [00:00<?, ?it/s]

Predictions - Min: 0.0000, Max: 1.0000, Mean: 0.0076
Unique prediction values: 59757
Prediction distribution:
(array([59347,   278,   119,    74,    52,    44,    50,    46,    46,
         103]), array([9.44209751e-06, 1.00008436e-01, 2.00007439e-01, 3.00006419e-01,
       4.00005430e-01, 5.00004411e-01, 6.00003421e-01, 7.00002432e-01,
       8.00001383e-01, 9.00000393e-01, 9.99999404e-01], dtype=float32))
Target distribution: [60100    59]
ROC curve points: 814
TPR range: 0.0000 to 0.8947
AUC: 0.9272
pAUC (competition metric): 0.1383
F1 Score: 0.1552

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60100
           1       0.09      0.46      0.16        59

    accuracy                           1.00     60159
   macro avg       0.55      0.73      0.58     60159
weighted avg       1.00      1.00      1.00     60159

Test AUC: 0.9272
Test pAUC (competition metric): 0.1383
Test F1 Score: 0.1552


In [ ]:
# Chunk 7: Test against validation set
import traceback
import torch

# Load the best hyperparameters
best_hyperparameters = torch.load('/content/current_best_hyperparameters.pth')
print("Loaded best hyperparameters:", best_hyperparameters)

# Prepare the validation dataset
vsurf_columns = ['clin_size_long_diam_mm', 'tbp_lv_H', 'tbp_lv_deltaLBnorm', 'tbp_lv_perimeterMM', 'tbp_lv_Hext']
val_vsurf_features = val_df[vsurf_columns].values

print("Validation VSURF features shape:", val_vsurf_features.shape)
print("Validation VSURF features type:", type(val_vsurf_features))
print("Validation VSURF features dtype:", val_vsurf_features.dtype)

# Create a DataLoader for the validation set
val_loader = get_dataloaders(val_df, local_hdf5_path, batch_size=64, vsurf_features=val_vsurf_features, is_training=False)

# Load the best model
best_model = setup_model(num_classes=1, vsurf_size=5, dropout_rate=best_hyperparameters['dropout_rate'])
best_model.load_state_dict(torch.load('best_model.pth'))
best_model = best_model.to(device)

# Make sure your model is in evaluation mode
best_model.eval()

# Evaluate the model on the validation set
print("Evaluating on Validation Set:")
try:
    for batch_idx, (inputs, vsurf_features, targets) in enumerate(val_loader):
        print(f"\nBatch {batch_idx}:")
        print("Inputs shape:", inputs.shape)
        print("VSURF features shape:", vsurf_features.shape)
        print("Targets shape:", targets.shape)

        inputs = inputs.to(device)
        vsurf_features = vsurf_features.to(device).float()

        try:
            outputs = best_model(inputs, vsurf_features)
            print("Outputs shape:", outputs.shape)
        except Exception as e:
            print(f"Error in model forward pass: {str(e)}")
            print(f"Traceback: {traceback.format_exc()}")

        if batch_idx == 0:  # Only print for the first batch
            break

    val_auc_score, val_pauc_score, val_f1_score = evaluate_model(best_model, val_loader, device)
    print(f"Validation AUC: {val_auc_score:.4f}")
    print(f"Validation pAUC (competition metric): {val_pauc_score:.4f}")
    print(f"Validation F1 Score: {val_f1_score:.4f}")
except Exception as e:
    print(f"Error during validation evaluation: {str(e)}")
    print(f"Traceback: {traceback.format_exc()}")

# Print model structure
print("\nModel structure:")
print(best_model)


<ipython-input-15-ec0b038caff6>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_hyperparameters = torch.load('/content/current_best_hyperparameters.pth')


Loaded best hyperparameters: {'learning_rate': 0.00014742482211184204, 'dropout_rate': 0.1, 'val_f1': 0.0821917808219178}
Validation VSURF features shape: (60159, 5)
Validation VSURF features type: <class 'numpy.ndarray'>
Validation VSURF features dtype: float64
Evaluating on Validation Set:


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-15-ec0b038caff6>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for 


Batch 0:
Inputs shape: torch.Size([64, 3, 224, 224])
VSURF features shape: torch.Size([64, 5])
Targets shape: torch.Size([64])
Outputs shape: torch.Size([64, 1])


Evaluating:   0%|          | 0/940 [00:00<?, ?it/s]

Predictions - Min: 0.0000, Max: 0.9999, Mean: 0.0069
Unique prediction values: 59759
Prediction distribution:
(array([59417,   245,   111,    81,    48,    42,    42,    44,    41,
          88]), array([1.2159118e-05, 1.0000154e-01, 1.9999091e-01, 2.9998028e-01,
       3.9996967e-01, 4.9995905e-01, 5.9994841e-01, 6.9993782e-01,
       7.9992718e-01, 8.9991659e-01, 9.9990594e-01], dtype=float32))
Target distribution: [60100    59]
ROC curve points: 808
TPR range: 0.0000 to 0.9341
AUC: 0.9469
pAUC (competition metric): 0.1544
F1 Score: 0.1899

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     60100
           1       0.12      0.51      0.19        59

    accuracy                           1.00     60159
   macro avg       0.56      0.75      0.59     60159
weighted avg       1.00      1.00      1.00     60159

Validation AUC: 0.9469
Validation pAUC (competition metric): 0.1544
Validation F1 Score: 0.1899

Mode